# Delta Live Tables, Materialized views

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE taxi_raw_records 
AS
SELECT *
FROM
STREAM(samples.nyctaxi.trips);

Name,Type
tpep_pickup_datetime,timestamp
tpep_dropoff_datetime,timestamp
trip_distance,double
fare_amount,double
pickup_zip,int
dropoff_zip,int


In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE taxi_raw_recordsonadb 
(CONSTRAINT valid_distance EXPECT (trip_distance > 0.0) ON VIOLATION DROP ROW ) 
AS SELECT
  *
FROM
  STREAM(samples.nyctaxi.trips);

Name,Type
tpep_pickup_datetime,timestamp
tpep_dropoff_datetime,timestamp
trip_distance,double
fare_amount,double
pickup_zip,int
dropoff_zip,int


In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE flagged_rides
AS SELECT
  date_trunc('week', tpep_pickup_datetime) as week,
  pickup_zip as zip,
  trip_distance, fare_amount
FROM
 STREAM(samples.nyctaxi.trips)
WHERE   ((pickup_zip = dropoff_zip AND fare_amount > 50) OR
        (trip_distance < 5 AND fare_amount > 50));


Name,Type
week,timestamp
zip,int
trip_distance,double
fare_amount,double


In [0]:
%sql
CREATE
OR REFRESH MATERIALIZED VIEW weekly_stats
AS SELECT
  date_trunc("week", tpep_pickup_datetime) as week,
  AVG(fare_amount) as avg_amount,
  AVG(trip_distance) as avg_distance
FROM
 live.taxi_raw_records
GROUP BY
  week
ORDER by week ASC;

Name,Type
week,timestamp
avg_amount,double
avg_distance,double


In [0]:
%sql
CREATE OR REPLACE MATERIALIZED VIEW top_m
AS SELECT
  weekly_stats.week,
  ROUND(avg_amount,2) as avg_amount, 
  ROUND(avg_distance,3) as avg_distance,
  fare_amount,trip_distance, zip 

  FROM live.flagged_rides
  LEFT JOIN live.weekly_stats ON flagged_rides.week = weekly_stats.week
  ORDER BY fare_amount DESC;

Name,Type
week,timestamp
avg_amount,double
avg_distance,double
fare_amount,double
trip_distance,double
zip,int
